<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumAlgorithms_VQA_QAOA_mkbahk_20250827.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install qiskit-algorithms==0.3.1
!pip install qiskit~=1.0
!pip install qiskit-aer

In [30]:
# pip install qiskit qiskit-algorithms  (필요 시)
from __future__ import annotations
from itertools import combinations
from typing import Dict, List, Tuple

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import Estimator, Sampler
from qiskit_algorithms.optimizers import COBYLA

In [31]:
# ----- 1) 그래프 정의 (무가중이면 weight=1) -----
n = 4
edges: List[Tuple[int,int,float]] = [
    (0,1,3.0), (0,2,1.0), (1,2,2.0), (1,3,1.0), (2,3,4.0)
]

In [32]:
# ----- 2) QAOA ansatz 구성 -----
def qaoa_ansatz(n: int, edges: List[Tuple[int,int,float]], p: int) -> Tuple[QuantumCircuit, ParameterVector, ParameterVector]:
    gammas = ParameterVector('γ', p)
    betas  = ParameterVector('β', p)
    qc = QuantumCircuit(n, name=f"QAOA_p{p}")
    # |+> 초기화
    for q in range(n):
        qc.h(q)
    # p layers
    for layer in range(p):
        γ = gammas[layer]
        # e^{-i γ sum w_ij Z_i Z_j} 구현: CX-RZ-CX (각 간선마다)
        for (i, j, w) in edges:
            qc.cx(i, j)
            qc.rz(2.0 * γ * w, j)   # ZZ(γ w)
            qc.cx(i, j)
        β = betas[layer]
        # 믹서: ∏ RX(2β)
        for q in range(n):
            qc.rx(2.0 * β, q)
    return qc, gammas, betas

In [33]:
# 비용 관측자(최소화용). 상수항은 버리고 +0.5 * Σ w_ij Z_i Z_j를 최소화
from qiskit.quantum_info import SparsePauliOp

def zz_cost_operator(n: int, edges: List[Tuple[int,int,float]]):
    labels = []
    coeffs = []
    for (i, j, w) in edges:
        z = ['I'] * n
        z[i] = 'Z'
        z[j] = 'Z'
        labels.append(''.join(reversed(z)))  # Qiskit은 리틀엔디안: 마지막 큐빗이 좌측
        coeffs.append(0.5 * w)               # +0.5 * w_ij * Z_i Z_j
    return SparsePauliOp.from_list(list(zip(labels, coeffs)))

In [34]:
op = zz_cost_operator(n, edges)
p = 2
qc, gammas, betas = qaoa_ansatz(n, edges, p)

In [35]:
# ----- 3) Estimator로 기대에너지 최소화 -----
estimator = Estimator()
def energy(theta):
    # theta = [γ_0..γ_{p-1}, β_0..β_{p-1}]
    vals = list(theta[:p]) + list(theta[p:])
    job = estimator.run([qc], [op], [vals])
    return job.result().values[0]

x0 = np.concatenate([0.8*np.ones(p), 0.3*np.ones(p)])  # 간단 초기값
opt = COBYLA(maxiter=150, tol=1e-3)
res = opt.minimize(energy, x0)
print("optimal params:", res.x, "  energy:", res.fun)

/tmp/ipython-input-2248908488.py:2: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


optimal params: [1.21668217 1.59705214 0.15326875 1.38186983]   energy: -2.5837953038171104


In [36]:
# ----- 4) 최적 파라미터로 샘플링해 최적 컷 찾기 -----
# 비트스트링에서 컷값 계산
def cut_value(bitstr: str, edges) -> float:
    z = [int(b) for b in bitstr[::-1]]  # Qiskit 비트 순서 보정
    total = 0.0
    for (i, j, w) in edges:
        total += w if z[i] != z[j] else 0.0
    return total

In [37]:
from qiskit.compiler import transpile
sampler = Sampler()
bound = qc.assign_parameters(list(res.x[:p]) + list(res.x[p:]))
# 측정 추가
meas = bound.copy()
meas.measure_all()
# 샘플
tqc = transpile(meas, optimization_level=1)
shots = 2000
pj = sampler.run(circuits=[tqc], parameter_values=[[]], shots=shots).result().quasi_dists[0]
# 최고 컷 찾기
best_str = max(pj.items(), key=lambda kv: cut_value(format(kv[0], f'0{n}b'), edges))[0]
best_bitstr = format(best_str, f'0{n}b')
best_cut = cut_value(best_bitstr, edges)
print("best bitstring:", best_bitstr, "  best cut:", best_cut)

best bitstring: 0101   best cut: 9.0


/tmp/ipython-input-2092284995.py:2: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
